![logos.png](attachment:logos.png)

<h1><center>
King Faisal University <br>
College of Computer Sciences and Information Technology <br>
CS324: Artificial Intelligence – Practical Class
</h1>


<strong>
<center>[0921 - 324]: [Artificial Intelligence]
<br><br>
<center>Section: [Male/Female]
<br><br><br><br>

<center>Lab [06]: [Local Search]
<br><br><br><br>

<center> Date: []
<center> Time: []
<br><br>


<center> Instructor: []
</strong>

# Introduction
The search algorithms that we have seen so far are designed to explore search spaces systematically. This systematicity is achieved by keeping one or more paths in memory and by recording which alternatives have been explored at each point along the path. When a goal is found, the path to that goal also constitutes a solution to the problem. In many problems, however, the path to the goal is irrelevant.

If the path to the goal does not matter, we might consider a different class of algorithms, ones that do not worry about paths at all. Local search algorithms operate using a single current node (rather than multiple paths) and generally move only to neighbors of that node. Typically, the paths followed by the search are not retained. Although local search algorithms are not systematic, they have two key advantages:

1.	They use very little memory—usually a constant amount.
2.	They can often find reasonable solutions in large or infinite (continuous) state spaces for which systematic algorithms are unsuitable.

In addition to finding goals, local search algorithms are useful for solving pure optimization problems, in which the aim is to find the best state according to an objective function. Examples of local search algorithms are:

1.	Hill-climbing search
2.	Simulated annealing
3.	Local beam search
4.	Genetic algorithms

However, in this lab, we will only focus on Hill Climbing algorithm and use an example to explain its implementation to solve a problem.



**Objectives: **
* Implement Local Search ALgorithm such as Hill-climbing to Solve Traveling salesman Problem.

**Tools/Software Requirement:**
*	Anaconda Navigator

**Lab Activity Description:**

Part 1: Implementing Informed Search Algorithm
* Solving Traveling Salesman using Hill-Climbing algorithm.

Part 2: Lab Task.
* Student discusses their observation  in the implementation of Hill-climbing algorithm in solving a problem.
* Student solves programming questions using Hill Climbing Algorithm.

# Part 1: Implementing Local Search Algorithms

## 1.1 Hill-Climbing Algorithm

Hill climbing is a mathematical optimization procedure whose goal is to discover the best solution to a problem with a (big) number of alternatives.

The best way to explain the algorithm (and optimization in general) is to use an example. We will use the Traveling salesman problem who needs to visit a number of cities exactly once before returning to the first. We have the distances between each pair of cities and need to identify the shortest path between them. As you might expect, a specific Traveling salesman problem has a huge number of different solutions (routes); the goal is to discover the best (i.e. shortest) solution.

### Solving the Problem

Hill climbing tries to find the best solution to this problem by starting out with a random solution, and then generate neighbours: solutions that only slightly differ from the current one. If the best of those neighbours is better (i.e. shorter) than the current one, it replaces the current solution with this better solution. It then repeats the pattern by again creating neighbours. If at some point no neighbour is better than the current solution, it returns the then current solution.

In [66]:
#Program imports
import random #For random solution generator


### A. Create a Travelling salesman Problem

Let's start with a code example of the Traveling salesman problem. When you think about it, such an instantiation should be a list of cities, each of which provides information about the distances between them. Naturally, the distance between each city and itself is zero, and the distance between cities A and B is the same as the distance between cities B and A. This yields a list containing n lists of size n (where n equals 4 in this case):

In [67]:
#As we can see, the distance between the first and third cities is, of course, 0;
#but, the distance between the first and third cities is 500.
tsp = [[0, 400, 500, 300],
       [400, 0, 300, 500],
       [500, 300, 0, 400],
       [300, 500, 400, 0]]

### B. Create a random solution generator

Hill climbing must begin with a random solution to our Traveling salesman problem in order to work. It can then produce neighboring solutions and begin the optimization process from there.

The solution to the problem of the traveling salesman could be as simple as a list of all city identifiers in the order in which the salesman should visit them. Every city must be visited only once. Let's start by making a list of all city identifiers (named cities in the code below), and then pick a city at random from that list and add it to our solution repeatedly. The range function in Python is great for making cities since it makes a range of all values from 0 to the parameter given, which in this case equals the length of the issue, because each city is represented by a single item in the problem.

In [68]:
#each city can only be visited once, we remove its identification from the city identifier
#list once it has been included to our solution.

def randomSolution(tsp):
    cities = list(range(len(tsp)))
#     print(cities)
#     print('#'*10)
    solution = []
    for i in range(len(tsp)):
        randomCity = cities[random.randint(0, len(cities) - 1)]
        solution.append(randomCity)
#         print(cities, 'before')
        cities.remove(randomCity)
#         print(cities, 'after')
#         print(solution)
    return solution

# print(randomSolution(tsp))

### C. Create a function calculating the length of a route

We need a method to calculate the length of a certain solution since we want our Hill Climber to locate the shortest option. This function requires the Traveling salesman problem (information on distances between cities) and the solution to which the route length is required.

We should iterate over a solution and use the tsp argument to add the distance to each new city to our overall route length because a solution is a list of all cities in a certain order. When i equals 0, the iterator i "visits" each city, so i-1 is "at" the previous or last city. As a result, solution[i] represents the current city, while solution[i-1] represents the previous one. Then we use the tsp to calculate the distance between these cities, which we then add to the entire route length (routeLength).

In [69]:
def routeLength(tsp, solution):
#     print(solution)
    routeLength = 0
    for i in range(len(solution)):
#         print("#"*20)
#         print("solution[i - 1]" ,solution[i - 1])
#         print("solution[i]",solution[i])
#         print("tsp[solution[i - 1]] [solution[i]]",tsp[solution[i - 1]] [solution[i]])
        routeLength += tsp[solution[i - 1]] [solution[i]]
#         print(routeLength)
    return routeLength
print(routeLength(tsp, randomSolution(tsp)))
    

1800


### D. Create a function generating all neighbours of a solution

Hill climbing works in part by creating all of the solutions that are neighboring to the current one. The term "neighboring solution" refers to a solution that is only slightly different from the existing one. It's also important to note that a neighbor must still be a correct solution: every city must be visited exactly once. We can do both by making a neighbor in the following way:

1. Copy the current solution and switch the locations of two cities! This manner, we can come up with a somewhat different but still correct solution.
2. Create two for loops, one nested in the other, both iterating the current solution, because we want to create all neighbors to a solution and need to have cities trade places. Because changing cities A and B is the same as swapping cities A and B, our second loop just has to loop over the cities that the first loop hasn't yet looped over.
3. Within the second loop, we build a swapped-city neighbor and add it to our list of neighbors.

In [70]:
def getNeighbours(solution):
#     print(solution)
    neighbours = []
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
         
            neighbour = solution.copy()
#             print('before')
#             print("location i", neighbour[i])
#             print("location j", neighbour[j])
            neighbour[i] = solution[j]
            neighbour[j] = solution[i]
#             print('after')
#             print("location i", neighbour[i])
#             print("location j", neighbour[j])
            neighbours.append(neighbour)
#             print("final", neighbours)
    print("solution",solution)

    return neighbours
# print(getNeighbours(randomSolution(tsp)))

### E. Create a function finding the best neighbour

Now that we have a function that generates all of a solution's neighbors, it's time to develop one that finds the best of them. This is a simple function: it sets bestNeighbour to the first neighbour in the list of neighbours (and bestRouteLength to its route length), then iterates over all neighbours, updating both bestNeighbour and bestRouteLength when a neighbor has a shorter route length. This is how the best neighbor is discovered.

In [71]:
def getBestNeighbour(tsp, neighbours):
    print("neighbours", neighbours)
    print("tsp", tsp)
    bestRouteLength = routeLength(tsp, neighbours[0])
    bestNeighbour = neighbours[0]
    print("best route length", bestRouteLength)
    print("bestNeighbour ", bestNeighbour)
    for neighbour in neighbours:
        currentRouteLength = routeLength(tsp, neighbour)
        print("current route length", currentRouteLength)
        if currentRouteLength < bestRouteLength:
            bestRouteLength = currentRouteLength
            bestNeighbour = neighbour
            print("current best route length: ", bestRouteLength)
            print("bestNeighbour: ", bestNeighbour)
    return bestNeighbour, bestRouteLength
# getBestNeighbour(tsp, )

### F. Create the Hill climbing algorithm

To begin, we generate a random solution and determine its path length. After that, we construct neighboring solutions and choose the best one. As long as the best neighbor is better than the existing solution, we repeat the process, with the current solution being updated with the best neighbor each time. We return the current solution after this process completes (and its route length).

In [72]:
def hillClimbing(tsp):
    currentSolution = randomSolution(tsp)
    currentRouteLength = routeLength(tsp, currentSolution)
    neighbours = getNeighbours(currentSolution)
    bestNeighbour, bestNeighbourRouteLength = getBestNeighbour(tsp, neighbours)
    while bestNeighbourRouteLength < currentRouteLength:
        currentSolution = bestNeighbour
        currentRouteLength = bestNeighbourRouteLength
        neighbours = getNeighbours(currentSolution)
        bestNeighbour, bestNeighbourRouteLength = getBestNeighbour(tsp, neighbours)

    return currentSolution, currentRouteLength

### G. Let Try it!

In Section A, the example of a Traveling salesman problem that is "rectangular": we have four cities, each in the corner of a rectangular form. The rectangle's long side is 400 kilometers (or whatever unit you like), while the short side is 300 kilometers. The diagonal is now 500 kilometers long. The shortest paths, it seems evident, travel along the sides of this rectangle, making the shortest route 2 x 400 + 2 x 300 = 1400 kilometers long.

In [73]:
#Try to run this multiple times.
print(hillClimbing(tsp))

solution [1, 0, 3, 2]
neighbours [[0, 1, 3, 2], [3, 0, 1, 2], [2, 0, 3, 1], [1, 3, 0, 2], [1, 2, 3, 0], [1, 0, 2, 3]]
tsp [[0, 400, 500, 300], [400, 0, 300, 500], [500, 300, 0, 400], [300, 500, 400, 0]]
best route length 1800
bestNeighbour  [0, 1, 3, 2]
current route length 1800
current route length 1400
current best route length:  1400
bestNeighbour:  [3, 0, 1, 2]
current route length 1600
current route length 1600
current route length 1400
current route length 1800
([1, 0, 3, 2], 1400)


### G.1 Create a problem generator

The problem is straightforward, with only four cities to consider. Let's add a problem generator to our code that can generate larger challenges for our Hill Climber to solve, and see how well it works.

To create a problem generator that is appropriate for our situation. Let's create a single-argument function called nCities, which indicates the number of cities we want in our Traveling salesman issue. The issue generator can then just loop over nCities times, each time producing a new list of distances (distances). Except where the value is already known: when the loop is adding the distance of the city to itself (which should be zero), and when it is adding a value it has already calculated, such a distance list can be filled with random numbers.This is the case when the algorithm adds the distance from city B to city A after previously adding the distance from city A to city B, despite the fact that these distances should be the same. In comment 1 and 2 from the code below this criteria is met.

In [10]:
def problemGenerator(nCities):
    tsp = []
    for i in range(nCities):
        distances = []
        for j in range(nCities):
            if j == i:
                distances.append(0)
            elif j < i: #comment 1
                distances.append(tsp[j][i]) #comment 2
            else:
                distances.append(random.randint(10, 1000))
        tsp.append(distances)
    return tsp

### G.2 Use the problem generator to create a big problem

Let's see how our Hill Climber performs on larger challenges now that we have a problem generator.

Note: Change the value of ct to define the number of cities.

The tsp has now been assigned a problem from the problem generator: one with ten cities, as you can see. The Hill Climber is called ten times with this problem to see if it returns the same quality solution each time. If it doesn't, we know the Hill Climber isn't always performing at its best.

In [14]:
ct = 10
tsp = problemGenerator(ct)
for i in range(ct):
  print(hillClimbing(tsp))

tsp [[0, 362, 900, 581, 821, 760, 465, 547, 296, 447], [362, 0, 362, 161, 628, 258, 358, 211, 395, 981], [900, 362, 0, 430, 401, 139, 622, 751, 493, 660], [581, 161, 430, 0, 222, 799, 223, 58, 979, 760], [821, 628, 401, 222, 0, 779, 500, 339, 643, 958], [760, 258, 139, 799, 779, 0, 515, 611, 645, 219], [465, 358, 622, 223, 500, 515, 0, 589, 86, 186], [547, 211, 751, 58, 339, 611, 589, 0, 953, 254], [296, 395, 493, 979, 643, 645, 86, 953, 0, 656], [447, 981, 660, 760, 958, 219, 186, 254, 656, 0]]
currentSolution [2, 4, 9, 0, 1, 7, 8, 6, 5, 3]
currentRouteLength 5162
neighbours [[4, 2, 9, 0, 1, 7, 8, 6, 5, 3], [9, 4, 2, 0, 1, 7, 8, 6, 5, 3], [0, 4, 9, 2, 1, 7, 8, 6, 5, 3], [1, 4, 9, 0, 2, 7, 8, 6, 5, 3], [7, 4, 9, 0, 1, 2, 8, 6, 5, 3], [8, 4, 9, 0, 1, 7, 2, 6, 5, 3], [6, 4, 9, 0, 1, 7, 8, 2, 5, 3], [5, 4, 9, 0, 1, 7, 8, 6, 2, 3], [3, 4, 9, 0, 1, 7, 8, 6, 5, 2], [2, 9, 4, 0, 1, 7, 8, 6, 5, 3], [2, 0, 9, 4, 1, 7, 8, 6, 5, 3], [2, 1, 9, 0, 4, 7, 8, 6, 5, 3], [2, 7, 9, 0, 1, 4, 8, 6, 5, 3], 

# Part 2: Lab Task

### Lab Task 1
Using the Code in G.2 execute the code and use different number of cities such as 10, 20 and 30. Based on the result discuss your observation about the result. Is hill-climbing ideal in solving larger problems?Why does the result changes? Also, explain how does local maxima will effect hill-climbing algorithm result.

### Type your answer below

### Lab Task 2

Despite the challenges of solving the Traveling Salesman Problem, it has application across different industries. For example. in the last mile delivery used in Vehicle Routing Problem. Efficient solutions discovered through the TSP are being employed. The movement of goods from a transportation center, such as a depot or warehouse, to the end customer's preferred delivery method is referred to as last mile delivery. Last-mile delivery is the most expensive part of the supply chain. This is why companies seek to keep the cost of last-mile delivery as low as possible.

The Vehicle Routing Problem (VRP) is everywhere, and solving it is critical in helping to facilitate the movement of goods and services from one place to another.One of this problem is, what order should the vehicles visit those stops to minimize the cost (last mile delivery)

Problem

ABC company is given you a task to determine the minimum last mile delivery cost and the route that a driver can follow using the destination in each town listed below (denoted in list).

since ABC company does not have a record of their yearly delivery transactions, you are about to similuate a 1 year (360 days) delivery transaction using the given list of destination and determine the minimum last mile delivery cost and the route to follow.

Note: Since delivery cars can start in any city its is important to take note that the generated route will only be a basis for the company to keep track on the cost based on the path that is supposed to be taken by the driver based on their location.

Example:

Given with this three result example with different starting point:

Route [0, 2, 7, 4, 1, 5, 9, 3, 8, 6]
Cost: 1242

Route [2, 7, 4, 1, 5, 9, 3, 8, 6, 0]
Cost 1242

Route [5, 9, 3, 8, 6, 0, 2, 7, 4, 1]
Cost 1242

Each of them has a minimum cost 1242 based on 360 days simulation but each one of the has a different starting point, meaning if a driver is currently residing in city 0, the driver can take route 0 - 2 - 7 - 4 - 1 - 5 - 9 - 3- 8 -6, but if the driver is residing in city 2, the driver can take 2 - 7 - 4 - 1 - 5 - 9 - 3 8 - 6 - 0.

Destination List

destination = [[0, 85, 80, 474, 59, 177, 87, 308, 773, 502],
       [85, 0, 801, 770, 40, 93, 231, 729, 979, 848],
       [80, 801, 0, 510, 650, 430, 995, 230, 655, 767],
       [474, 770, 510, 0, 178, 62, 805, 564, 53, 62],
       [59, 40, 650, 178, 0, 632, 610, 53, 839, 667],
       [177, 93, 430, 62, 632, 0, 897, 116, 317, 134],
       [87, 231, 995, 805, 610, 897, 0, 490, 410, 479],
       [308, 729, 230, 564, 53, 116, 490, 0, 79, 874],
       [773, 979, 655, 53, 839, 317, 410, 79, 0, 655],
       [502, 848, 767, 62, 667, 134, 479, 874, 655, 0]]


Write your solution here
